<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_t81_559_class10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative AI
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 10 Assignment: StreamLit**

**Student Name: Your Name**

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
  from google.colab import drive, userdata
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google CoLab")
except:
  print("Note: not using Google CoLab")
  COLAB = False

# Assignment Submission Key - Was sent you first week of class.
# If you are in both classes, this is the same key.
if COLAB:
  # For Colab, add to your "Secrets" (key icon at the left)
  key = userdata.get('T81_559_KEY')
else:
  # If not colab, enter your key here, or use an environment variable.
  # (this is only an example key, use yours)
  key = ""

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai openai streamlit

Mounted at /content/drive
Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io
from typing import List, Union

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 10.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.

def submit(
    data: List[Union[pd.DataFrame, PIL.Image.Image]],
    key: str,
    course: str,
    no: int,
    source_file: str = None
) -> None:
    if source_file is None and '__file__' not in globals():
        raise Exception("Must specify a filename when in a Jupyter notebook.")
    if source_file is None:
        source_file = __file__

    suffix = f'_class{no}'
    if suffix not in source_file:
        raise Exception(f"{suffix} must be part of the filename.")

    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb', '.py']:
        raise Exception(f"Source file is {ext}; must be .py or .ipynb")

    with open(source_file, "rb") as file:
        encoded_python = base64.b64encode(file.read()).decode('ascii')

    payload = []
    for item in data:
        if isinstance(item, PIL.Image.Image):
            buffered = io.BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG': base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif isinstance(item, pd.DataFrame):
            payload.append({'CSV': base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
        else:
            raise ValueError(f"Unsupported data type: {type(item)}")

    response = requests.post(
        "https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key': key},
        json={
            'payload': payload,
            'assignment': no,
            'course': course,
            'ext': ext,
            'py': encoded_python
        }
    )

    if response.status_code == 200:
        print(f"Success: {response.text}")
    else:
        print(f"Failure: {response.text}")

# Assignment Instructions

Write a StreamLit application that prompts the user to enter a comma-separated list of ICD-10 codes and displays a table with the corresponding SNOMED codes with the human-readable name of the code. Use gpt-4o-mini to perform this translation.

SNOMED (Systematized Nomenclature of Medicine) and ICD-10 (International Classification of Diseases, 10th Revision) are comprehensive systems for coding medical conditions and diseases. ICD-10, developed by the World Health Organization (WHO), is primarily used for diagnostic purposes in clinical and billing contexts. In contrast, SNOMED, managed by SNOMED International, provides a more detailed and granular representation of clinical information. Most ICD-10 codes map to SNOMED codes, allowing for more standardized exchange of medical data across different healthcare systems. This mapping enables healthcare providers to document patient conditions precisely, enhancing data interoperability and patient care.

Some examples of ICD-10 codes include, with the corresponding SNOWMED code are:
* R10.10: Upper abdominal pain
    * SNOMED: 21522001 - Upper abdominal pain
* R10.30: Lower abdominal pain
    * SNOMED: 5008001 - Lower abdominal pain
* E11.9: Type 2 diabetes mellitus without complications
    * SNOMED: 44054006 - Diabetes mellitus type 2
* Z20.828: Contact with and (suspected) exposure to other viral communicable diseases
    * SNOMED: 443684005 - Exposure to communicable disease
* U07.1: COVID-19, virus identified
    * SNOMED: 840539006 - COVID-19
* J06.9: Acute upper respiratory infection, unspecified
    * SNOMED: 54150009 - Acute upper respiratory infection
* I10: Essential (primary) hypertension
    * SNOMED: 38341003 - Essential hypertension
* M54.5: Low back pain
    * SNOMED: 279039007 - Low back pain
* R51: Headache
    * SNOMED: 25064002 - Headache

You can use these to test your program.

Your program should look something like this:

![Assignment 10](https://data.heatonresearch.com/images/wustl/app_genai/assignments/t81-559-10.jpg)

### Sample Code

In [ ]:
%%writefile app.py
import streamlit as st

st.write("Hello World")

Writing app.py


Next, we obtain the password for our StreamLit server we are about to launch.

In [ ]:
!curl https://loca.lt/mytunnelpassword

34.81.158.212

We launch the StreamLit server and obtain its URL. You will need the above password when you access the URL it gives you.

In [ ]:
!streamlit run app.py server1 &>/content/logs.txt &
!npx --yes localtunnel --port 8501

⠙⠹⠸⠼⠴your url is: https://dry-groups-wait.loca.lt
^C


Submit your assignment (make sure to stop your StreamLit server first).

In [ ]:
# Add your solution here, put your results into submit_df

%%writefile app.py
import streamlit as st
from langchain_core.messages import HumanMessage
from llm_util import open_llm
import pandas as pd
import json
import sys
from PIL import Image
from typing import List, Union
import base64, os, requests, io


# You must also identify your source file.  (modify for your local setup)
# file='/content/drive/MyDrive/Colab Notebooks/assignment_yourname_t81_559_class10.ipynb'  # Google CoLab

key = "HgCchttxVS1LGxD5VHKeW93yo96aH3Hp4gC0FHOt"

file= '/content/drive/My Drive/Colab Notebooks/assignment_ZhijiangLi_class10.ipynb'


arguments = sys.argv
if len(arguments) != 2:
    print("Please specify the llm to use as the first argument")
    st.stop()
else:
    profile = arguments[1]

st.title("ICD-10 to SNOMED Code Translator")

if "chat" not in st.session_state:
    st.session_state.chat = open_llm(profile)

icd_input = st.text_input("Enter ICD-10 codes (comma-separated):")

if st.button("Translate"):
    if not icd_input.strip():
        st.warning("Please enter at least one ICD-10 code.")
    else:
        prompt = f"""
You are a medical coding assistant. Given the following list of ICD-10 codes: {icd_input},
return a JSON array where each entry has:
- "ICD-10 Code"
- "SNOMED Code"
- "Description"

Only return valid JSON. Example:
[
  {{
    "ICD-10 Code": "R10.10",
    "SNOMED Code": "21522001",
    "Description": "Upper abdominal pain"
  }}
]
"""
        response = st.session_state.chat.invoke([HumanMessage(content=prompt)])
        try:
            results = json.loads(response.content)
            df = pd.DataFrame(results)
            st.dataframe(df)
        except Exception:
            st.error("⚠️ Could not parse response as JSON.")
            st.code(response.content)

# Write llm_util.py
%%writefile llm_util.py
import yaml

def load_yaml(file_path):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

def get_class(class_path):
    module_path, class_name = class_path.rsplit(".", 1)
    module = __import__(module_path, fromlist=[class_name])
    return getattr(module, class_name)

def open_llm(server_name):
    config = load_yaml("llms.yaml")
    for server in config["servers"]:
        if server["name"] == server_name:
            class_path = server["class"]
            clazz = get_class(class_path)
            params = {k: v for k, v in server.items() if k not in ["class", "name"]}
            return clazz(**params)
    raise ValueError(f"Server '{server_name}' not found")

# Write llms.yaml
%%writefile llms.yaml
servers:
  - name: server1
    class: langchain_openai.ChatOpenAI
    model: gpt-4o-mini
    temperature: 0

# # Get password for tunneling
# !curl https://loca.lt/mytunnelpassword

# # Run Streamlit server
# !streamlit run app.py server1 &>/content/logs.txt &

# # Create tunnel to public URL
# !npx --yes localtunnel --port 8501



# df = pd.DataFrame([
#     {"ICD-10 Code": "R10.10", "SNOMED Code": "21522001", "Description": "Upper abdominal pain"},
#     {"ICD-10 Code": "E11.9", "SNOMED Code": "44054006", "Description": "Type 2 diabetes mellitus without complications"},
# ])


# Note: no dataframe to submit, just your notebook
submit(source_file=file,data=[],key=key,course='t81-559',no=10)

Overwriting app.py


In [ ]:
!curl https://loca.lt/mytunnelpassword
!streamlit run app_class10.py server1 &>/content/logs.txt &
!npx --yes localtunnel --port 8501


34.71.14.249⠙⠹⠸⠼⠴your url is: https://seven-mangos-bet.loca.lt


In [ ]:
import requests
import pandas as pd
import base64
import os

def list_submits(key):
    r = requests.post("https://api.heatonresearch.com/wu/submit",
                      headers={'x-api-key': key},
                      json={'course':'t81-558'})
    if r.status_code == 200:
        print("Success: \n{}".format(r.text))
    else:
        print("Failure: {}".format(r.text))

def display_submit(key,no):
    r = requests.post("https://api.heatonresearch.com/wu/submit",
                      headers={'x-api-key': key},
                      json={'course':'t81-558', 'assignment':no})
    if r.status_code == 200:
        print("Success: \n{}".format(r.text))
    else:
        print("Failure: {}".format(r.text))


In [ ]:
key = "HgCchttxVS1LGxD5VHKeW93yo96aH3Hp4gC0FHOt"

list_submits(key)

Failure: {"message":"Forbidden"}


In [ ]:
# Show one assignment, by number.

display_submit(key,1)

Failure: {"message":"Forbidden"}
